# Shifting then Tilting HG(0,0)

***
The effect of a tilt to an already misaligned beam of fundamental mode:

$
	\sum_{n,m} u_{nm}(x,y,0)= u_{00(tilt)}(x,y,0)+ \frac{a}{w_0}u_{10(tilt)}(x,y,0) \; .
$

The first term is verified to be

$
	u_{00 (tilt)} \approx u_{00}(x,y,0) + i  \frac{\pi w_{0} \alpha}{\lambda}u_{10} \; .
$

The second term approximates to

$
		\frac{a}{w_0} u_{tilt(1,0)}(x,y,0) 			 
			\approx  
			\frac{a}{w_0} 
			u_{10} 
			 \left[
			1 + i \frac{2 \pi x \alpha}{\lambda} \right] \;.
$

***
Therefore:

$
		u(x,y,0)=
			\left[
				1+
				i \frac{ \pi a \alpha}{ \lambda}
			\right]
			u_{00}
			+
			\left[
				i  \frac{\pi w_{0} \alpha}{\lambda}
			+
			\frac{a}{w_{0}}
			\right]			
			u_{10}
			+
			i  
				\frac{\sqrt{2} \pi a \alpha}{ \lambda}
			u_{20}	.
$





In [1]:
import PauLisa as pl, numpy as np, matplotlib.pyplot as plt, cmath as cm
from scipy import integrate
from mpmath import quadgl

In [2]:
j = [[2,1,3],[1,1,1]]
print(j[0][0])


2


In [4]:
j = [1,2]

k = pl.modes_from_1d_array_LV(j,j)
print(k)


[[1.+1.j 2.+2.j]]


In [ ]:
an_array = np.array(((1, 2,1),(3, 4,1)))

a = map(tuple, an_array)
t = tuple(a)

print(a)
print(t)
print(type(t))
m = pl.modes(j)
#print(type(m))
pl.show_modes(m)

In [ ]:
#Redefining and naming initial parameters
high_res_plane = pl.Plane(-6e-3,7e-3,1000,-1e-3,1e-3,3) #just used to calculate peak, higher precision
plane = pl.Plane(-2e-3,2e-3,1000,-2e-3,2e-3,1000)
params = pl.Params(wavelength=1.064e-06, w0=1e-3, z0=0)
a = 1e-6 #shift
alpha = 1e-6 #tilt angle

#wavelength
lam = params.wavelength #wavelength
w0 = params.w0 #spot size
E0 = 1   #E field

#scales of each mode
c00_ref = 1
c00 = (1+ (np.pi * a * alpha/ lam)*1j)
c10 = ((a/w0)+ (np.pi*w0*alpha/lam)*1j)
c20 = (np.sqrt(2)*np.pi*a*alpha/lam)*1j


print(c00,c10,c20)

#create modes received and reference (rec and ref)
modes_rec = pl.modes((0,0,c00),(1,0,c10),(2,0,c20))
modes_ref = pl.modes((0,0,c00_ref))


#calculate amp. and phase
result_rec = pl.calculate(params, plane, modes_rec, z=0)
result_ref = pl.calculate(params,plane,modes_ref, z=0)



## Comparing ref and rec 

In [ ]:
#compare shift to approx.
result_peak = pl.calculate(params, high_res_plane, modes_rec, z=0)
act_shift = pl.PeakInt(result_peak).x[0]

pred_shift = (c10.real*w0)/abs((c00))
print("predicted shift:",pred_shift,'[m]')
print("actual shift:",act_shift,'[m]')

#plot
pl.intslicex(0, result_rec, result_ref, labels=['rec','ref'])



In [ ]:
#compare alpha to approx.
theta=lam/pl.pi/w0

pred_alpha = theta*abs(c10.imag)

phase_comb=(result_rec.phase[1,:])
phase_diff=(phase_comb[-1]-phase_comb[0])
phase_ang = phase_diff/(plane.getX()[-1]-plane.getX()[0])
act_alpha = phase_ang/2/np.pi*lam

#plot
pl.phaseslicex(0, result_rec, result_ref,labels=['rec','ref'])

#results
print("predicted alpha:",pred_alpha,'[rad]')
print("actual alpha:",act_alpha,'[rad]')

## Add ref and rec

In [ ]:
#combined (comb)
modes_comb = pl.modes((0,0,c00+c00_ref),(1,0,c10),(2,0,c20))
amp_comb = result_rec.amp+result_ref.amp

result_row = len(amp_comb)
result_col = len(amp_comb[0])
phase_comb = np.zeros((result_row,result_col), dtype=float)
for r in range(result_row):
    for c in range(result_col):
        phase_comb[r,c] = (np.angle(amp_comb[r][c]))

result_comb = pl.Result(params,plane,modes_comb,0,amp_comb,phase_comb)

## Photodector 
PD power:
\begin{equation}
	P_{pd}= \int_{- \infty}^{\infty}\int_{- \infty}^{\infty} E^*E \; dxdy,
\end{equation}

Beat note from argument of overlap integral:

\begin{equation}
	\Phi= arg \lbrace \int_{- \infty}^{\infty}\int_{- \infty}^{\infty} U_{ref}(x,y)^*U_{prb}(x,y) \rbrace \; dxdy,
\end{equation}


\begin{equation}
	arg \lbrace 
    \int\int dxdy \:
    U_{ref}(x,y)^*U_{prb}(x,y) 
    \rbrace = 
    arg
    \lbrace
		|E_{0}|^2 
        \int\int dxdy \;
			e^{i  \Delta \phi }
				\left[
					1 + i \frac{\pi a \alpha }{\lambda}
				\right]
					\rbrace 
		\;.
\end{equation}

optopy:

\begin{equation}
	\Phi= arg \lbrace sum( U_{ref}(x,y)^*U_{prb}(x,y) ) \rbrace 
\end{equation}

In [ ]:
f00 = lambda y, x: result_ref.amp*result_rec.amp

In [ ]:
f = lambda y, x: (pl.amplitude(params,x,y,0,modes_rec))*np.conjugate((pl.amplitude(params,x,y,0,modes_ref)))

\begin{equation}
C_{nmn'm'}^{R} = \int_{0}^{\infty} dx\int_{-\infty}^{\infty} dy \; U_{00(LO)}^*(x,y,z) U_{nm(RX)}(x,y,z)
\end{equation}

In [ ]:
cr = quadgl(f, [plane.ymin,plane.ymax],[0,plane.xmax])

\begin{equation}
C_{nmn'm'}^{L} = \int_{-\infty}^{0} dx\int_{-\infty}^{\infty} dy \; U_{00(LO)}^*(x,y,z) U_{nm(RX)}(x,y,z)
\end{equation}

In [ ]:
cl = quadgl(f, [plane.ymin,plane.ymax],[plane.xmin,0])

In [ ]:
print(cr,cl)

In [ ]:
print(cr-cl)

\begin{equation}
	\phi_R = 
		arg (C_{nmn'm'}^{R})
 \end{equation}

In [ ]:
phi_r = cm.phase(cr)
print(phi_r)

\begin{equation}
	\phi_L = 
		arg (C_{nmn'm'}^{L})
 \end{equation}

In [ ]:
phi_l = cm.phase(cl)
print(phi_l)

\begin{equation}
    \Delta \phi = \phi_R - \phi_L
\end{equation}

In [ ]:
print (phi_r-phi_l)